In [8]:
import cv2
import torch
import math
import numpy as np
import pytesseract
import pandas as pd
from collections import defaultdict
from IPython.display import display, Image
from ultralytics import YOLO
import time

# Set the Tesseract OCR path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to display images in JupyterLab
def show_image(img):
    _, img_encoded = cv2.imencode('.png', img)
    display(Image(data=img_encoded.tobytes()))

# Euclidean Distance Tracker with IOU for more accurate tracking
class EuclideanDistTracker:
    def __init__(self):
        self.center_points = {}  # id -> (cx, cy, w, h)
        self.id_count = 0
        self.lost_count = {}
        self.timeout = 30  # Frames before an object is considered lost

    def update(self, objects_rect):
        objects_bbs_ids = []
        current_ids = set()

        def compute_iou(boxA, boxB):
            # Calculate IOU (Intersection Over Union) between two boxes
            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
            yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

            interArea = max(0, xB - xA) * max(0, yB - yA)
            boxAArea = boxA[2] * boxA[3]
            boxBArea = boxB[2] * boxB[3]
            iou = interArea / float(boxAArea + boxBArea - interArea)
            return iou

        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2
            same_object_detected = False

            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                size_diff_w = abs(w - pt[2])
                size_diff_h = abs(h - pt[3])
                
                # Adding IOU check
                iou = compute_iou((x, y, w, h), (pt[0] - pt[2] // 2, pt[1] - pt[3] // 2, pt[2], pt[3]))

                distance_threshold = max(100, 0.7 * max(w, h))
                size_threshold = max(50, 0.5 * (pt[2] + pt[3]))

                if dist < distance_threshold and size_diff_w < size_threshold and size_diff_h < size_threshold and iou > 0.3:
                    self.center_points[id] = (cx, cy, w, h)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    current_ids.add(id)
                    break

            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy, w, h)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Handle lost objects
        for id in list(self.center_points.keys()):
            if id not in current_ids:
                self.lost_count[id] = self.lost_count.get(id, 0) + 1
                if self.lost_count[id] > self.timeout:
                    del self.center_points[id]
                    del self.lost_count[id]

        return objects_bbs_ids


# Load YOLOv8 model
model = YOLO('yolov8l.pt')

# Video Capture
cap = cv2.VideoCapture(r'VID_20241004_154027.mp4')

# Set the starting timestamp in milliseconds (if needed)
# start_time_ms = 102000
# end_time_ms = 122000
# cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)






In [9]:
tracker = EuclideanDistTracker()

# Video Writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec
output_video_path = 'numberplate_output.avi'
fps = 30  # Frames per second
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize counters and sets to track unique object IDs and entry/exit times
tracked_objects = {}
car_count = 0
bike_count = 0
animal_count = 0
person_count = 0
unique_car_ids = set()
unique_bike_ids = set()
unique_animal_ids = set()
unique_person_ids = set()

# Data storage for Excel
license_data = []

# Define labels to count
vehicle_labels = ['car', 'motorcycle', 'bicycle', 'truck']
animal_labels = ['dog', 'cat', 'cow']
person_label = 'person'  # Label for detecting people

# Create a dataframe to log vehicle and person entry/exit times
df = pd.DataFrame(columns=['Object Type', 'Object ID', 'License Plate', 'Entry Time', 'Exit Time'])

while True:
    # current_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
    # if current_time_ms > end_time_ms:
    #     break

    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    detections = []
    detected_labels = []

    for result in results:
        for detection in result.boxes:
            x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy().astype(int)
            conf = detection.conf[0].item()
            cls = detection.cls[0].item()
            label = model.names[int(cls)]

            if label in vehicle_labels + animal_labels + [person_label] and conf > 0.4:
                detections.append([x1, y1, x2 - x1, y2 - y1])
                detected_labels.append(label)

    # Object tracking
    boxes_ids = tracker.update(detections)

    # Draw detection and tracking results
    for idx, box_id in enumerate(boxes_ids):
        x, y, w, h, object_id = box_id
        label = detected_labels[idx] if idx < len(detected_labels) else "Unknown"

        # Track entry and exit times
        if object_id not in tracked_objects:
            tracked_objects[object_id] = {'entry': time.time(), 'license_plate': '', 'type': label}

            # Count the object only when it's first detected
            if label == 'car' or label == 'truck':
                if object_id not in unique_car_ids:
                    car_count += 1
                    unique_car_ids.add(object_id)  # Add object to unique IDs set
            elif label in ['motorcycle', 'bicycle']:
                if object_id not in unique_bike_ids:
                    bike_count += 1
                    unique_bike_ids.add(object_id)
            elif label in animal_labels:
                if object_id not in unique_animal_ids:
                    animal_count += 1
                    unique_animal_ids.add(object_id)
            elif label == person_label:
                if object_id not in unique_person_ids:
                    person_count += 1
                    unique_person_ids.add(object_id)

        # Set the color based on the type of object
        if label == 'car' or label == 'truck':
            color = (0, 255, 0)  # Green for cars
        elif label in ['motorcycle', 'bicycle']:
            color = (255, 0, 0)  # Blue for bikes
        elif label in animal_labels:
            color = (0, 255, 255)  # Yellow for animals
        elif label == person_label:
            color = (255, 0, 255)  # Magenta for people
        else:
            color = (255, 255, 255)  # Default color for unknown

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

        # Put label and ID above the bounding box
        cv2.putText(frame, f'{label} ID: {object_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # License Plate Detection (for cars and bikes)
        if label == 'car' or label == 'motorcycle' or label == 'truck':
            # Crop the region of interest (ROI) for the license plate
            roi = frame[y:y+h, x:x+w]
            gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            # Use Tesseract to extract text from the ROI
            license_plate = pytesseract.image_to_string(gray_roi, config='--psm 8')
            license_plate = license_plate.strip()

            # Store the detected license plate along with object ID
            if license_plate:
                tracked_objects[object_id]['license_plate'] = license_plate

    # Display counts on the frame
    cv2.putText(frame, f'Cars: {car_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Bikes: {bike_count}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Animals: {animal_count}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'People: {person_count}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the frame to the output video file
    out.write(frame)


0: 384x640 1 toilet, 1350.5ms
Speed: 13.3ms preprocess, 1350.5ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1527.7ms
Speed: 11.4ms preprocess, 1527.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1741.2ms
Speed: 11.0ms preprocess, 1741.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1395.0ms
Speed: 23.0ms preprocess, 1395.0ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1249.6ms
Speed: 3.4ms preprocess, 1249.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1325.6ms
Speed: 6.3ms preprocess, 1325.6ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1304.6ms
Speed: 17.2ms preprocess, 1304.6ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1353.7ms
Speed: 25.1ms preprocess, 1353.7ms 

KeyboardInterrupt: 

In [ ]:
for object_id, obj_info in tracked_objects.items():
    obj_info['exit'] = time.time()
    new_row = pd.DataFrame([{
        'Object Type': obj_info['type'],
        'Object ID': object_id,
        'License Plate': obj_info['license_plate'],
        'Entry Time': obj_info['entry'],
        'Exit Time': obj_info['exit']
    }])
    df = pd.concat([df, new_row], ignore_index=True)


# Release video objects and close files
cap.release()
out.release()

# Save the data to Excel
df.to_excel('vehicle_entry_exit_times_night.xlsx', index=False)